# 🧪 A/B Testing Framework for Ad Spend Optimization

**Goal:** Design and implement a statistical framework to test budget reallocation strategies.

This notebook covers:
- **Power Analysis** - determine required sample sizes
- **Test Design** - randomization and control strategies  
- **Statistical Testing** - significance testing for conversion rates
- **Confidence Intervals** - estimate effect sizes
- **Sequential Testing** - early stopping criteria

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, chi2_contingency, fisher_exact
from statsmodels.stats.power import ttest_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
import warnings
warnings.filterwarnings('ignore')

print("🧪 A/B Testing framework loaded!")

class ABTestFramework:
    """
    A comprehensive A/B testing framework for ad spend optimization
    """
    
    def __init__(self, alpha=0.05, power=0.8):
        self.alpha = alpha  # Significance level
        self.power = power  # Statistical power
        
    def calculate_sample_size(self, baseline_rate, minimum_detectable_effect, 
                            alternative='two-sided'):
        """
        Calculate required sample size for proportion test
        
        Parameters:
        - baseline_rate: Current conversion rate
        - minimum_detectable_effect: Minimum effect size to detect (e.g., 0.1 for 10% relative improvement)
        - alternative: 'two-sided' or 'larger'
        """
        
        effect_size = baseline_rate * minimum_detectable_effect
        
        # Calculate required sample size
        if alternative == 'two-sided':
            z_alpha = norm.ppf(1 - self.alpha/2)
            z_beta = norm.ppf(self.power)
        else:
            z_alpha = norm.ppf(1 - self.alpha)
            z_beta = norm.ppf(self.power)
        
        # Using normal approximation for proportion test
        p1 = baseline_rate
        p2 = baseline_rate + effect_size
        p_pooled = (p1 + p2) / 2
        
        numerator = (z_alpha * np.sqrt(2 * p_pooled * (1 - p_pooled)) + 
                    z_beta * np.sqrt(p1 * (1 - p1) + p2 * (1 - p2)))**2
        
        denominator = (p2 - p1)**2
        
        n_per_group = numerator / denominator
        
        return int(np.ceil(n_per_group))
    
    def run_proportion_test(self, control_conversions, control_visitors, 
                          treatment_conversions, treatment_visitors):
        """
        Run a two-proportion z-test
        """
        
        # Calculate conversion rates
        control_rate = control_conversions / control_visitors
        treatment_rate = treatment_conversions / treatment_visitors
        
        # Run the test
        count = np.array([treatment_conversions, control_conversions])
        nobs = np.array([treatment_visitors, control_visitors])
        
        z_stat, p_value = proportions_ztest(count, nobs)
        
        # Calculate confidence interval for the difference
        diff = treatment_rate - control_rate
        se_diff = np.sqrt((control_rate * (1 - control_rate) / control_visitors) + 
                         (treatment_rate * (1 - treatment_rate) / treatment_visitors))
        
        ci_lower = diff - 1.96 * se_diff
        ci_upper = diff + 1.96 * se_diff
        
        # Calculate relative improvement
        relative_improvement = ((treatment_rate - control_rate) / control_rate) * 100
        
        results = {
            'control_rate': control_rate,
            'treatment_rate': treatment_rate,
            'difference': diff,
            'relative_improvement_pct': relative_improvement,
            'z_statistic': z_stat,
            'p_value': p_value,
            'is_significant': p_value < self.alpha,
            'confidence_interval': (ci_lower, ci_upper)
        }
        
        return results
    
    def bayesian_ab_test(self, control_conversions, control_visitors,
                        treatment_conversions, treatment_visitors,
                        prior_alpha=1, prior_beta=1):
        """
        Bayesian A/B test using Beta-Binomial model
        """
        
        # Posterior parameters for control
        control_alpha_post = prior_alpha + control_conversions
        control_beta_post = prior_beta + control_visitors - control_conversions
        
        # Posterior parameters for treatment  
        treatment_alpha_post = prior_alpha + treatment_conversions
        treatment_beta_post = prior_beta + treatment_visitors - treatment_conversions
        
        # Sample from posteriors
        n_samples = 100000
        control_samples = np.random.beta(control_alpha_post, control_beta_post, n_samples)
        treatment_samples = np.random.beta(treatment_alpha_post, treatment_beta_post, n_samples)
        
        # Calculate probability that treatment > control
        prob_treatment_better = np.mean(treatment_samples > control_samples)
        
        # Calculate expected lift
        expected_lift = np.mean((treatment_samples - control_samples) / control_samples) * 100
        
        # Credible intervals
        lift_samples = (treatment_samples - control_samples) / control_samples * 100
        lift_ci = np.percentile(lift_samples, [2.5, 97.5])
        
        results = {
            'probability_treatment_better': prob_treatment_better,
            'expected_lift_pct': expected_lift,
            'lift_credible_interval': lift_ci,
            'control_posterior_mean': control_alpha_post / (control_alpha_post + control_beta_post),
            'treatment_posterior_mean': treatment_alpha_post / (treatment_alpha_post + treatment_beta_post)
        }
        
        return results
    
    def sequential_test(self, data, method='obrien_fleming'):
        """
        Sequential testing with early stopping
        """
        # Implementation for sequential testing would go here
        # This is a simplified version
        pass
    
    def plot_power_analysis(self, baseline_rate, effect_sizes, sample_sizes):
        """
        Plot power curves for different effect sizes and sample sizes
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Power vs Effect Size
        powers = []
        for effect in effect_sizes:
            n = self.calculate_sample_size(baseline_rate, effect)
            power = self.calculate_power(baseline_rate, baseline_rate * (1 + effect), n)
            powers.append(power)
        
        ax1.plot(np.array(effect_sizes) * 100, powers, 'b-', linewidth=2)
        ax1.axhline(y=0.8, color='r', linestyle='--', alpha=0.7, label='80% Power')
        ax1.set_xlabel('Minimum Detectable Effect (%)')
        ax1.set_ylabel('Statistical Power')
        ax1.set_title('Power vs Effect Size')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Sample Size vs Effect Size
        sample_sizes_needed = [self.calculate_sample_size(baseline_rate, effect) for effect in effect_sizes]
        
        ax2.plot(np.array(effect_sizes) * 100, sample_sizes_needed, 'g-', linewidth=2)
        ax2.set_xlabel('Minimum Detectable Effect (%)')
        ax2.set_ylabel('Required Sample Size (per group)')
        ax2.set_title('Sample Size vs Effect Size')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def calculate_power(self, p1, p2, n):
        """Calculate statistical power for given parameters"""
        effect_size = abs(p2 - p1) / np.sqrt(p1 * (1 - p1))
        power = zt_ind_solve_power(effect_size, nobs1=n, alpha=self.alpha)
        return power

# Initialize the framework
ab_framework = ABTestFramework(alpha=0.05, power=0.8)
print("✅ A/B Testing framework initialized!")